In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *
from glove import *
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from textblob import TextBlob
from tweets_ops import *
from url_helpers import analyze_url
from matching import *
from textstat.textstat import textstat
import urllib, bs4

pd.options.mode.chained_assignment = None

import warnings
warnings.simplefilter("ignore", category=UserWarning)

In [2]:
df6 = pd.read_csv(cache_dir+'article_details_v6.tsv', sep='\t')
df7 = pd.read_csv(cache_dir+'article_details_v7.tsv', sep='\t').fillna(0).drop('Alexa Rank', axis=1)
df7['Author Signature'] = df6['authors'].apply(lambda x: len(eval(x)) != 0)


In [3]:
for a in ['#Likes', '#Replies', 'Title Clickbaitness', 'Betweenness Centrality', 'Degree Centrality', 
          'In Degree Centrality', 'Out Degree Centrality', '#Retweets', 'Tweets Shelf Life', '#Users Countries', 
          '#Followers', '#Users Friends', '#Users Tweets', 'STS', 'Readability', '#Quotes', '#Person Quotes', 
          '#Scientific Mentions', '#Weasel Quotes', 'Personalized PageRank', 'Article Word Count']:
    
    df7[a] = pd.cut(df7[a], 5, labels=False)
    df7[a] = df7[a].apply(lambda x: (int(x)+1) * '★' + (5-(int(x)+1)) * '☆')

In [4]:
for a in ['Replies Polarity', 'Replies Subjectivity', 'Replies Stance', 'Tweets Polarity', 'Tweets Subjectivity', 
          'Title Subjectivity', 'Title Polarity']:
    
    df7[a] = pd.cut(df7[a], 5, labels=False)
    if a == 'Replies Stance':
        df7[a] = 4-df7[a]
    df7[a] = df7[a].apply(lambda x: {0:'☹☹', 1:'☹', 2:'😐', 3:'☺', 4:'☺☺'}[x])


In [5]:
df7['Author Signature'] = df7['Author Signature'].apply(lambda x: '✔' if x==True else '✘')

In [6]:
df7.to_csv(cache_dir+'article_details_v8.tsv', sep='\t', index=None)

In [7]:
df7 = pd.read_csv(cache_dir+'crispr.tsv', sep='\t').merge(df7, left_on='article', right_on='url')
dfa = pd.read_csv(cache_dir+'alexa-crispr.tsv', sep='\t')

In [8]:
df7 = df7.merge(dfa)
df7['Alexa Rank'] = pd.cut(df7['Alexa Rank'], 5, labels=False)

In [9]:
df7['Alexa Rank'] = 5-df7['Alexa Rank']

In [10]:
df7['Alexa Rank'] = df7['Alexa Rank'].apply(lambda x: int(x) * '★' + (5-(int(x))) * '☆')
df7 = df7.drop('url', axis=1)

In [11]:
df7

,article,paper,title,#Likes,#Replies,Title Clickbaitness,Betweenness Centrality,Degree Centrality,In Degree Centrality,Out Degree Centrality,...,Readability,Title Subjectivity,Title Polarity,#Quotes,#Person Quotes,#Scientific Mentions,#Weasel Quotes,Author Signature,Article Word Count,Alexa Rank
0,http://dailyhealthpost.com/bee-venom-hiv-cure/,http://www.nature.com/articles/srep22555,"""Cure for AIDS? Bee Venom Destroys HIV Cells, ...",★☆☆☆☆,★☆☆☆☆,★★☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,☹☹,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✘,★☆☆☆☆,★★★★★
1,http://futurism.com/scientists-just-surgically...,http://www.cdc.gov/hiv/statistics/overview/ata...,Scientists Just Surgically Removed HIV DNA Fro...,★☆☆☆☆,★☆☆☆☆,★★★★☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,😐,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
2,http://futurism.com/scientists-move-remarkably...,http://clinicaltrials.gov/ct2/show/NCT02788175,Scientists Move Remarkably Close to a Function...,★☆☆☆☆,★☆☆☆☆,★★★☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,☺,☺☺,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
3,http://futurism.com/the-fda-just-approved-a-fi...,http://www.fda.gov/NewsEvents/Newsroom/PressAn...,The FDA Just Approved a First-Of-Its-Kind Drug...,★☆☆☆☆,★☆☆☆☆,★★★☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,☹☹,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
4,http://genomealberta.ca/livestock/old-mcdonald...,http://www.nature.com/news/crispr-gene-editing...,Old McDonald's Gene-edited Farm,★☆☆☆☆,★☆☆☆☆,★★☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,☹☹,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✘,★☆☆☆☆,★★★☆☆
5,http://joshmitteldorf.scienceblog.com/2016/12/...,http://www.nature.com/news/crispr-gene-editing...,"""From Santa Diego, a Jolla Xmas Gift""",★☆☆☆☆,★☆☆☆☆,★★★★☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,☹☹,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★☆☆☆☆
6,http://labiotech.eu/hiv-cure-abivax-review/,http://www.nature.com/articles/srep22555,An HIV Cure by 2020? A Review of the Future of...,★☆☆☆☆,★☆☆☆☆,★★★☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,☹☹,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✘,★☆☆☆☆,★★★★★
7,http://mashable.com/2017/05/03/scientists-cris...,http://www.sciencedirect.com/science/article/p...,Scientists can eliminate HIV infection in mice...,★☆☆☆☆,★☆☆☆☆,★★★☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,☺☺,☺,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
8,http://medium.com/@TwistBioscience/top-moments...,http://www.nature.com/news/crispr-gene-editing...,Top Moments in Synthetic Biology 2016,★☆☆☆☆,★☆☆☆☆,★★★☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,😐,☺,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★
9,http://newatlas.com/hiv-cure-crispr-cas9-mice/...,http://www.sciencedirect.com/science/article/p...,Closer to a cure: CRISPR cuts HIV from its cel...,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,...,★★★★★,☹☹,😐,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,★☆☆☆☆,✔,★☆☆☆☆,★★★★★


In [12]:
#df7['Alexa Rank'] = df7['url'].apply(alexa)
df7.to_csv(cache_dir+'crispr_v2.tsv', sep='\t', index=None)

In [ ]:
import subprocess
import os
import string

def is_clickbait(titles):
        cb=[]
        os.chdir('../lib/clickbait')
        for t in titles:
            t = ''.join([c for c in str(t) if c in string.printable])
            out, err = subprocess.Popen(['venv/bin/python', 'src/detect.py', t], stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()
            try:
                cb.append(float(re.findall('\d*\.?\d+', str(out))[0]))
            except:
                cb.append(0.0)
        os.chdir('../../src')
        return cb

In [ ]:
outlet1 = 'dailymail.co.uk'
outlet2 = 'nytimes.com'
outlet3 = 'theatlantic.com'
outlet4 = 'washingtonpost.com'

# G = read_graph(cache_dir+'diffusion_graph/pruned_graph_v3.tsv')
# pagerank = nx.pagerank(G.reverse())
# betweenness_centrality = nx.betweenness_centrality(G)
# degree_centrality = nx.degree_centrality(G)
# in_degree_centrality = nx.in_degree_centrality(G)
# out_degree_centrality = nx.out_degree_centrality(G)

In [ ]:
outlet = outlet1
df = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t')


df[df.apply(lambda x: analyze_url(x['url'])[0]==outlet, axis=1)].to_csv(cache_dir+outlet+'.tsv', sep='\t', index=None)

aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v3.tsv', cache_dir+outlet+'.tsv', cache_dir+outlet+'.tsv')
df = pd.read_csv(cache_dir+outlet+'.tsv', sep='\t').fillna(0)

for s in ['full', 'paragraph', 'sentence']:
    train = pd.read_csv(cache_dir+'similarity_model/train_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    train = train[train.related==True]
    test = pd.read_csv(cache_dir+'similarity_model/test_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    pd.concat([train, test]).to_csv(cache_dir+'similarity_model/exp2/'+s+'.tsv', sep='\t', index=None)
    
test_similarity_model(cache_dir+'similarity_model/exp2/', cache_dir+'similarity_model/rf_model.sav', cache_dir+'similarity_model/exp2/results.tsv')
sim = pd.read_csv(cache_dir+'similarity_model/exp2/results.tsv', sep='\t').drop('paper', axis=1).groupby('article').max().reset_index()
df = df.merge(sim, left_on='url', right_on='article').drop('article', axis=1)

df['readability'] = df['full_text'].apply(lambda x: textstat.flesch_reading_ease(x))
df['title_subjectivity'] = df['title'].apply(lambda x: TextBlob(x).subjectivity)
df['title_polarity'] = df['title'].apply(lambda x: TextBlob(x).polarity)
df['title_clickbaitness'] = df['title'].apply(is_clickbait)

df = pd.concat([df.drop(['quote_indicators'], axis=1), df['quote_indicators'].apply(lambda x: pd.Series(eval(x)))], axis=1)


df['has_author'] = df['authors'].apply(lambda x: len(eval(x)) != 0)

df['pageRank'] = df['url'].apply(lambda x: pagerank[x])
df['betweenness_centrality'] = df['url'].apply(lambda x: betweenness_centrality[x])
df['degree_centrality'] = df['url'].apply(lambda x: degree_centrality[x])
df['in_degree_centrality'] = df['url'].apply(lambda x: in_degree_centrality[x])
df['out_degree_centrality'] = df['url'].apply(lambda x: out_degree_centrality[x])

df['word_count'] = df['full_text'].apply(lambda x: len(re.findall(r'\w+', x)))

df['alexa_rank']=df['url'].apply(lambda x: bs4.BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url="+str(x)).read(), "xml").find("REACH")['RANK'])


df.url = df.url.apply(lambda x: analyze_url(x)[0])

df = df[['url', 'likes', 'replies_count', 'title_clickbaitness',
    'betweenness_centrality', 'degree_centrality', 'in_degree_centrality', 'out_degree_centrality',
    'replies_mean_polarity', 'replies_mean_subjectivity', 'retweets',
    'stance', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
    'tweets_time_delta', 'users_countries', 'users_median_followers',
    'users_median_friends', 'users_median_tweets', 'related', 'readability',
    'title_subjectivity', 'title_polarity', 'count_all_quotes',
    'count_PER_quotes', 'count_ORG_quotes', 'count_unnamed_quotes',
    'has_author', 'pageRank', 'word_count', 'alexa_rank']]

df.columns = ['url', '#Likes', '#Replies', 'Title Clickbaitness',
    'Betweenness Centrality', 'Degree Centrality', 'In Degree Centrality', 'Out Degree Centrality',
    'Replies Polarity', 'Replies Subjectivity', '#Retweets',
    'Replies Stance', 'Tweets Polarity', 'Tweets Subjectivity',
    'Tweets Shelf Life', '#Users Countries', '#Followers',
    '#Users Friends', '#Users Tweets', 'STS', 'Readability',
    'Title Subjectivity', 'Title Polarity', '#Quotes',
    '#Person Quotes', '#Scientific Mentions', '#Weasel Quotes',
    'Author Signature', 'Personalized PageRank', 'Article Word Count', 'Alexa Rank']

df.to_csv(cache_dir+outlet+'.tsv', sep='\t', index=None)

In [ ]:
df = pd.read_csv(cache_dir+'article_details_v5.tsv', sep='\t').head(10)

aggregate_tweet_details(cache_dir+'diffusion_graph/pruned_graph_v3.tsv', cache_dir+'tweet_details_v3.tsv', cache_dir+'article_details_v5.tsv', cache_dir+'article_details_v6.tsv')
df = pd.read_csv(cache_dir+'article_details_v6.tsv', sep='\t')

for s in ['full', 'paragraph', 'sentence']:
    train = pd.read_csv(cache_dir+'similarity_model/train_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    train = train[train.related==True]
    test = pd.read_csv(cache_dir+'similarity_model/test_pairs_v2_'+s+'.tsv', sep='\t').merge(df[['url']], left_on='article', right_on='url').drop('url', axis=1)
    pd.concat([train, test]).to_csv(cache_dir+'similarity_model/exp2/'+s+'.tsv', sep='\t', index=None)
    
test_similarity_model(cache_dir+'similarity_model/exp2/', cache_dir+'similarity_model/rf_model.sav', cache_dir+'similarity_model/exp2/results.tsv')
sim = pd.read_csv(cache_dir+'similarity_model/exp2/results.tsv', sep='\t').drop('paper', axis=1).fillna(0).groupby('article').max().reset_index()
df = df.merge(sim, left_on='url', right_on='article').drop('article', axis=1)

df['readability'] = df['full_text'].apply(lambda x: textstat.flesch_reading_ease(str(x)))
df['title_subjectivity'] = df['title'].apply(lambda x: TextBlob(str(x)).subjectivity)
df['title_polarity'] = df['title'].apply(lambda x: TextBlob(str(x)).polarity)

In [ ]:
df['title_clickbaitness'] = is_clickbait(df['title'].tolist())

In [ ]:
df = pd.concat([df.drop(['quote_indicators'], axis=1), df['quote_indicators'].apply(lambda x: pd.Series(eval(x)))], axis=1)


df['has_author'] = ~(df['authors'].isnull() | (df['authors'] == ''))

df['pageRank'] = df['url'].apply(lambda x: pagerank[x])
df['betweenness_centrality'] = df['url'].apply(lambda x: betweenness_centrality[x])
df['degree_centrality'] = df['url'].apply(lambda x: degree_centrality[x])
df['in_degree_centrality'] = df['url'].apply(lambda x: in_degree_centrality[x])
df['out_degree_centrality'] = df['url'].apply(lambda x: out_degree_centrality[x])

df['word_count'] = df['full_text'].apply(lambda x: len(re.findall(r'\w+', x)))

df['alexa_rank']=df['url'].apply(lambda x: bs4.BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url="+str(x)).read(), "xml").find("REACH")['RANK'])


df.url = df.url.apply(lambda x: analyze_url(x)[0])

df = df[['url', 'likes', 'replies_count', 'title_clickbaitness',
    'betweenness_centrality', 'degree_centrality', 'in_degree_centrality', 'out_degree_centrality',
    'replies_mean_polarity', 'replies_mean_subjectivity', 'retweets',
    'stance', 'tweets_mean_polarity', 'tweets_mean_subjectivity',
    'tweets_time_delta', 'users_countries', 'users_median_followers',
    'users_median_friends', 'users_median_tweets', 'related', 'readability',
    'title_subjectivity', 'title_polarity', 'count_all_quotes',
    'count_PER_quotes', 'count_ORG_quotes', 'count_unnamed_quotes',
    'has_author', 'pageRank', 'word_count', 'alexa_rank']]

df.columns = ['url', '#Likes', '#Replies', 'Title Clickbaitness',
    'Betweenness Centrality', 'Degree Centrality', 'In Degree Centrality', 'Out Degree Centrality',
    'Replies Polarity', 'Replies Subjectivity', '#Retweets',
    'Replies Stance', 'Tweets Polarity', 'Tweets Subjectivity',
    'Tweets Shelf Life', '#Users Countries', '#Followers',
    '#Users Friends', '#Users Tweets', 'STS', 'Readability',
    'Title Subjectivity', 'Title Polarity', '#Quotes',
    '#Person Quotes', '#Scientific Mentions', '#Weasel Quotes',
    'Author Signature', 'Personalized PageRank', 'Article Word Count', 'Alexa Rank']

df.to_csv(cache_dir+'article_details_v6.tsv', sep='\t', index=None)

In [ ]:
df = pd.concat([pd.read_csv(cache_dir+outlet1+'.tsv', sep='\t'), pd.read_csv(cache_dir+outlet3+'.tsv', sep='\t')])
y = df[['url']].values
X = np.array(df.drop('url', axis=1).values, dtype=np.float32)
X = (X - X.mean(axis=0)) / (X.std(axis=0)+1e-9)

from sklearn.feature_selection import f_classif
_, result = f_classif(X,y)

d = {c:r for c, r in zip(df.columns[1:], result)}
df = pd.DataFrame(sorted(d.items(), key=lambda kv: kv[1]))

df[0] = df.apply(lambda x: x[0]+'***' if float(x[1])<.005 else x[0]+'**' if float(x[1])<.01 else x[0]+'*' if float(x[1])<.05 else x[0], axis=1)

df13 = df.copy()

In [ ]:
df = pd.concat([df12, df13, df23, df123], axis=1).head(5)[0]
df.columns = ['NYTimes-DailyMail', 'TheAtlantic-DailyMail', 'NYTimes-TheAtlantic', 'NYTimes-TheAtlantic-DailyMail']

In [ ]:
df

In [ ]:
print(df.to_latex(index=False))